In [3]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 22 10:12:35 2017
TODO

# For batch generator and multiproc c.f.
    https://gist.github.com/tdeboissiere/195dde7fddfcf622a82a895b90d2c800
# SEE ATHENA >> /examples/research/cbb_keras_tensorflow.py

# https://keras.io/getting-started/sequential-model-guide/
# =============================================================================

# see for GPU  https://keras.io/utils/#multi_gpu_model

@author: i015225
"""

# In[Headless Server for faster img generation]
import matplotlib
matplotlib.use("Agg")

# In[139]:
#c=get_ipython()
#%config InlineBackend.figure_format = 'svg'
#%pprint #Toggle
# get_ipython().magic(u'matplotlib inline')

import pandas as pd
from pandas.api.types import is_bool_dtype, is_categorical, infer_dtype, is_categorical_dtype
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import AutoDateLocator, AutoDateFormatter, date2num
from matplotlib.dates import DateFormatter
import seaborn as sns

import time
from datetime import datetime, timedelta
import itertools
import statsmodels.api as sm
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import sys
from sklearn import svm
from sklearn.metrics import accuracy_score, make_scorer

# Import the scikit-learn function to confusion matrix.
from sklearn.metrics import confusion_matrix
# Import the scikit-learn function to compute error.
from sklearn.metrics import mean_squared_error

import matplotlib
from pandas.tseries.offsets import BDay

import keras
from keras import backend as K

#import tensorflow as tf
#import numpy
from keras import layers, models, optimizers
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers import LSTM, Dense, Dropout, Flatten, Embedding, TimeDistributed
from keras.wrappers.scikit_learn import KerasRegressor
from keras import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler ,quantile_transform
from sklearn.pipeline import Pipeline

from keras import callbacks

import threading
    
# In[Charts]:
 # -*- coding: utf-8 -*-
"""
Created on Tue Jun 26 15:04:00 2018

@author: i015225
"""


'\nCreated on Tue Jun 26 15:04:00 2018\n\n@author: i015225\n'

In [4]:

def img_gen_type(kind='scatter', n=5, grid=False, legend=False, xticks=False, yticks=False, border=False, draw_size=2.24, color=['black'], style=None, show=False):
    """ Generate a random chart exporting height array and pixel array
        use yticks=[] etc to hide. None to show default
        TODO: Add options for randomising scale, # bars, showing axis, grids, rotating labels, legends etc.
        TODO: drop Y-Axis labels
        TODO: change background
        TODO: axis ticks/label rotation
    """
    data = pd.DataFrame(data=np.random.rand(n,1), index=range(1,n+1), columns=['y'])
    #m,n = np.shape(data)
    fig, ax = plt.subplots(1, 1)
    if kind=='scatter':
        # TODO add s for size and c for color per point , perhaps colormap='viridis'
        data['x']= data.index
        data.plot(ax=ax, kind=kind, x='x', y='y' , figsize=(draw_size, draw_size), grid=grid, legend=legend, style=style, color=color, marker='.', s=3)
        data.drop(['x'], axis=1, inplace=True)
    else:
        data.plot(ax=ax, kind=kind , figsize=(draw_size, draw_size), grid=grid, legend=legend, style=style, color=color)

    if not xticks:
        x_axis = ax.axes.get_xaxis()
        x_axis.set_visible(False)
    if not yticks:
        #ax1 = plt.axes()
        y_axis = ax.axes.get_yaxis()
        y_axis.set_visible(False)
        
    if not border:   
        plt.axis('off') #https://stackoverflow.com/questions/40705614/hide-axis-label-only-not-entire-axis-in-pandas-plot
    fig.tight_layout()
    fig.canvas.draw()
    # grab the pixel buffer and dump it into a numpy array
    pixels = np.array(fig.canvas.renderer._renderer)[:,:,:3]/255
    
    if not show:
        plt.close()
    # print(pixels.shape)
    return pixels, data.values.ravel()


In [5]:
#################### Now make the data generator threadsafe ####################
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def next(self):
        with self.lock:
            return self.it.next()


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def img_gen_safe(chttype='bar', batch = 10): #TODO add kwargs list
    if chttype=='rand':
        # use a random integer to choose the chart type
        chttype = np.random.randint(0,3)
        chttype = ['bar','plot','scatter'][chttype]  #,'histo'

    while True:
        X=[]
        y=[]
        for b in range(batch):
            #do some drawing
            X_, y_ = img_gen_type(kind=chttype)

            X.append(X_)
            y.append(y_.T)

        X = np.array(X)
        y = np.stack(y)

        yield X, y
    #return X, y


In [6]:
def img_gen(chttype='bar', batch = 10): #TODO add kwargs list
    if chttype=='rand':
        # use a random integer to choose the chart type
        chttype = np.random.randint(0,3)
        chttype = ['bar','plot','scatter'][chttype]  #,'histo'

    while True:
        X=[]
        y=[]
        for b in range(batch):
            #do some drawing
            X_, y_ = img_gen_type(kind=chttype)

            X.append(X_)
            y.append(y_.T)

        X = np.array(X)
        y = np.stack(y)

        yield X, y

In [7]:

def train_generator( batch_size, shift_fraction=0.):
    gen = img_gen(chttype='scatter', batch = 5)  #add arguments here for color='rand', chttype=... etc.
    while True:
        x_batch, y_batch = next(gen)
        yield ([x_batch, y_batch], [y_batch, x_batch])
    return ([x_batch, y_batch], [y_batch, x_batch])


In [8]:
gen = img_gen(chttype='scatter', batch = 5)  #add arguments here for color='rand', chttype=... etc.
X,y = next(gen)#


In [9]:
#X,y=img_gen_type(kind='bar', show=True)
print(y)

[[0.26466733 0.69680911 0.74433061 0.77580667 0.06979797]
 [0.50128855 0.56583097 0.52116675 0.59887559 0.54529723]
 [0.5403671  0.48509047 0.75114016 0.04069109 0.62052449]
 [0.87449496 0.78116094 0.57559513 0.23030076 0.93131143]
 [0.94439522 0.40514459 0.33580496 0.55129404 0.41916742]]


In [10]:
# In[Build Keras Model CNN]:
def model_CNN(x_train, y_train, x_test=None, y_test=None, kwargs={}):
    """ Build but do not run a NN model in Keras framework
        train and test data requried to determine model types, dimensions etc.
        x_train, y_train, x_test, y_test, kwargs = self.X4, self.y, None, None, self.kwargs
    """
    """
    Notes on Input shape
    4D tensor with shape (batch_size, timesteps, features, `colors`).
    4D tensor with shape: (samples, rows, cols, channels)
    `channels_last` (default)
    Output 4D tensor with shape: (samples, new_rows, new_cols, filters)
    """
    ######## CNN for stocks
    # create and fit CNN
    # input_shape = StockDate x Lookback x Features
    from keras.layers import Conv2D, MaxPooling2D
    from keras.optimizers import SGD


    layers = kwargs.get('layers', 10 ) #TODO
    nodes = kwargs.get('nodes', None) #TODO

    if nodes is None or nodes==0 or nodes==[0]:
        nodes = [np.shape(x_train)[1]*3]
    elif isinstance(nodes, (int, np.integer)): # turn int to list
        nodes = [nodes]

    if layers > 1 and len(nodes) < layers:
        nodes = list(np.pad(nodes,[0,layers-len(nodes)], mode='constant',constant_values=nodes[-1]))

    ndim = np.max([2,len(np.shape(x_train))]) # Min 2D
    if ndim==2:
        input_shape=(x_train.shape[1],)
    elif ndim==3:
        input_shape=(x_train.shape[1],x_train.shape[2])
    elif ndim==4:
        input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])
    else:
        input_shape=x_train.shape[1:]
    if kwargs.get('learning_rate', False):
        lr = kwargs.get('learning_rate')
    else:
        lr = False

    if False:
        conv = (3, 3)
    else:
        conv = (2, 2)
        n_conv = 5

    if np.ndim(y_train)==1:
        n_out = 1 #e.g. forecast y as float, just 1 step ahead.
    else:
        n_out = np.shape(y_train)[1] #e.g. onehot encoded, or n-steps ahead.

    dropout = kwargs.get('dropout',0) # dropout rate between 0 and 1.
    #stateful = kwargs.get('stateful',True)
    actvn = 'relu' #kwargs.get('actvn','relu')
    actvl = kwargs.get('actvl','sigmoid')
    model=[]
    model = Sequential()  # https://keras.io/models/sequential/
    model.reset_states()
    # input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
    # this applies 32 convolution filters of size 3x3 each.
    model.add(Conv2D(n_conv, conv, activation=actvn, input_shape=input_shape))
    #model.add(Conv2D(n_conv, conv, activation=actvn))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout ))

    model.add(Conv2D(n_conv*2, conv, activation=actvn))
    #model.add(Conv2D(n_conv*2, conv, activation=actvn))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout ))

    model.add(Flatten())
    model.add(Dense(np.min(input_shape[1:-1]), activation=actvn))
    model.add(Dropout(dropout*2))
    model.add(Dense(n_out, activation=actvl))

    if hasattr(kwargs,'optimizer'):
        optimizer = kwargs['optimizer']
    elif lr:
        optimizer = SGD(lr=lr, decay=1e-6, momentum=0.01, nesterov=True)
    else:
        optimizer = 'Nadam' #keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

    if is_bool_dtype(y_train):
        model.compile(loss='binary_crossentropy', optimizer=optimizer)
    if is_categorical_dtype(y_train) or kwargs.get('onehot',False):
        #TODO Multiple Category
        model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    else:
        #model.compile(loss='mean_squared_error', optimizer=optimizer)
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[r2_keras])


    if kwargs.get('verbose',False) > 1:
        model.summary()
        print("Inputs: {}".format(model.input_shape))
        print("Outputs: {}".format(model.output_shape))
        print("Actual input: {}".format(x_train.shape))
        print("Actual output: {}".format(y_train.shape))
        print('Model Loss: ' + model.loss)

    # For compatability with other models;
    model.score = model.evaluate

    return model #self.model=model



In [11]:

def r2_keras(y_true, y_pred):
    # calculate R2 value (not built in :( )
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    #( 1 - SS_res/(SS_tot + K.epsilon()) )
    return ( 1 - SS_res/(SS_tot ) )


def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))


In [12]:

# In[Setup Config Variables]
verbose = 2  #True/False, extra verbose=2

config={}
#lagged return features
config['verbose'] = 3  #Single Int
#config['layers'] = 3  #Single Int
#config['nodes'] = [300*200*10, 300*200*5, 300*200]#[75, 50, 25, 10, 5] #ignored for 1 layer. size of output. None to match X[0], Int or list of dims per layer. [10, 5, 1]
#config['epochs'] = 10  # e.g. use 3*3 for 1st tstrain then 3 for tsroll (lower 'weight'? if less epochs)
#config['actvn'] = 'relu' #tanh'  #

config['Description'] = 'Digitize Bar Charts'
config['StartTime'] = time.time()

config['Model'] = "CNN"
config['iterations'] = 1000  #Single Int


class_names = ['False','True']  # Corresponds to False, True


# In[Setup args]:
class my_args():
    def __init__(self):
        self.lr = 0.01
        self.lr_decay  = 0.001
        self.lam_recon = 0.392  # , type=float,           help="The coefficient for the loss of decoder")
        self.routings = 3
        #return self
args = my_args()


In [13]:
# In[Build Keras Model ]:
if True:
    # Setup image generator
    gen = img_gen(chttype='scatter', batch = 500)  #add arguments here for color='rand', chttype=... etc.

    # Create sample data
    X, y = next(gen) #next is default

    # Build model
    model = model_CNN(X, y, x_test=None, y_test=None, kwargs=config)
    # Then produce evaluation data set or 1 off dataset

    print("Shapes of 1st Training sets: ", np.shape(X), np.shape(y))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 223, 223, 5)       65        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 5)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 5)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 10)      210       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 55, 10)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 55, 55, 10)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30250)             0         
__________

In [14]:

# In[Evaluate/Test Keras Simple]:
if True:
    if True:
        #model.evaluate(x_train_k, y_train_k, batch_size=1, verbose=1)
        mdlfit=[]
        mdlscr=[]
        mdlprd=[]

        # Make predictions.
        if True: #Use safe generator
            #gen is unsafe...?
            safe_gen = img_gen(chttype='scatter', batch = 500)  #add arguments here for color='rand', chttype=... etc.
            spare_gen = img_gen(chttype='scatter', batch = 50)
            third_gen = img_gen(chttype='scatter', batch = 50)
            
            i = 0
            while i < config.get('iterations', 3):
                i += 1
                #X, y = next(gen)
                if verbose: print('Training: Fit and Predicting for %s. Shapes: '%('img'), np.shape(X), np.shape(y))

                # Step 1
                m = model.fit_generator(safe_gen, steps_per_epoch=10, verbose=1, epochs=11)
                mdlfit.append(m) #, steps=100))

                # evaluate the model
                s = model.evaluate_generator(spare_gen, steps=10)
                mdlscr.append(s)
                #if verbose>1:print("%s: %.2f%%" % (model.metrics_names[0], scores[-1]*100))

            # Finished let's see
            #X_test,  y_test  = next(gen)

            #mdlprd = model.predict(X_test)   #, index=X.loc[x_train.index[L:]].index)
            mdlprd = model.predict_generator(generator=third_gen)
        else: # Single estimate
            X_train,  y_train  = next(gen)
            X_test,  y_test  = next(gen)
            mdlfit = model.fit(X_train, y_train, epochs=config.get('epochs'), batch_size=1, verbose=1, shuffle=False) #y.astype(int)
            mdlprd = model.predict(X_test)   #, index=X.loc[x_train.index[L:]].index)
            mdlscr = model.score(X_test, y_test)


Training: Fit and Predicting for img. Shapes:  (500, 224, 224, 3) (500, 5)
Epoch 1/11
10/10 [==============================] - 263s 26s/step - loss: 0.0845 - r2_keras: -0.0216
Epoch 2/11
10/10 [==============================] - 259s 26s/step - loss: 0.0805 - r2_keras: 0.0361
Epoch 3/11
10/10 [==============================] - 252s 25s/step - loss: 0.0840 - r2_keras: -0.0140
Epoch 4/11
10/10 [==============================] - 256s 26s/step - loss: 0.0840 - r2_keras: -0.0011
Epoch 5/11
10/10 [==============================] - 254s 25s/step - loss: 0.1054 - r2_keras: -0.2603
Epoch 6/11
10/10 [==============================] - 257s 26s/step - loss: 0.0797 - r2_keras: 0.0383
Epoch 7/11
10/10 [==============================] - 252s 25s/step - loss: 0.0835 - r2_keras: -0.0012
Epoch 8/11
10/10 [==============================] - 258s 26s/step - loss: 0.0847 - r2_keras: -0.0010
Epoch 9/11
10/10 [==============================] - 254s 25s/step - loss: 0.0846 - r2_keras: -4.6141e-04
Epoch 10/11
10

RuntimeError: Your generator is NOT thread-safe.Keras requires a thread-safe generator when`use_multiprocessing=False, workers > 1`.For more information see issue #1638.

In [16]:
import os
os.getcwd()

'/mnt/mesos/sandbox/data'

In [ ]:

## In[Save Keras Model]:

hist = pd.DataFrame([m.history['loss']  for m in mdlfit],columns=['loss'])
hist['oos'] = np.array(mdlscr)[:,0]
print('training loss',hist)
print('oos loss')
hist.plot()
